### Gemini API 준비

In [ ]:
# 패키지 설치
!pip install -q -U google-generativeai

In [ ]:
from google.colab import userdata
import google.generativeai as genai

# 환경 변수 준비(좌측 하단의 열쇠 아이콘으로 GOOGLE_API_KEY 설정)
GOOGLE_API_KEY=userdata.get("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

### 자동 함수 호출(Automatic Function Calling)

In [ ]:
def add(a:float, b:float):
    """returns a + b."""
    return a+b

def subtract(a:float, b:float):
    """returns a - b."""
    return a-b

def multiply(a:float, b:float):
    """returns a * b."""
    return a*b

def divide(a:float, b:float):
    """returns a / b."""
    return a*b

In [ ]:
# 모델 초기화시 함수 목록 지정
model = genai.GenerativeModel(
    model_name="models/gemini-1.5-flash",
    tools=[add, subtract, multiply, divide]
)

In [ ]:
# 채팅 준비
chat = model.start_chat(
    enable_automatic_function_calling=True
)

In [ ]:
# 함수를 활용한 질의 응답
response = chat.send_message(
    "저는 57마리의 고양이를 키우고 있고, 각 44개의 손싸개를 가지고 있습니다. 손싸개는 총 몇 개 일까요?"
)
response.text

'고양이의 손싸개는 총 2508개입니다. \n'

In [ ]:
# 대화 이력 확인
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("--")

user -> [{'text': '저는 57마리의 고양이를 키우고 있고, 각 44개의 손싸개를 가지고 있습니다. 손싸개는 총 몇 개 일까요?'}]
--
model -> [{'function_call': {'name': 'multiply', 'args': {'a': 57.0, 'b': 44.0}}}]
--
user -> [{'function_response': {'name': 'multiply', 'response': {'result': 2508.0}}}]
--
model -> [{'text': '고양이의 손싸개는 총 2508개입니다. \n'}]
--


### 도구 설정

In [ ]:
from google.generativeai.types import content_types
from collections.abc import Iterable

def tool_config_from_mode(mode: str, fns: Iterable[str] = ()):
    """함수 호출 모드와 허용하는 함수명으로 도구 설정"""
    return content_types.to_tool_config(
        {"function_calling_config": {"mode": mode, "allowed_function_names": fns}}
    )

In [ ]:
# 대화 이력 삭제
chat.history.clear()

# 추론 실행
response = chat.send_message(
    "저는 57마리의 고양이를 키우고 있고, 각 44개의 손싸개를 가지고 있습니다. 손싸개는 총 몇 개 일까요?",
    tool_config=tool_config_from_mode("none") # NONE
)

# 대화 이력 확인
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("--")

user -> [{'text': '저는 57마리의 고양이를 키우고 있고, 각 44개의 손싸개를 가지고 있습니다. 손싸개는 총 몇 개 일까요?'}]
--
model -> [{'text': '57마리의 고양이가 각각 44개의 손싸개를 가지고 있다면, 총 손싸개 수는 다음과 같습니다.\n\n57 마리의 고양이 * 44개의 손싸개/고양이 = **2508개의 손싸개** \n'}]
--


In [ ]:
# 대화 이력 삭제
chat.history.clear()

# 추론 실행
response = chat.send_message(
    "저는 57마리의 고양이를 키우고 있고, 각 44개의 손싸개를 가지고 있습니다. 손싸개는 총 몇 개 일까요?",
    tool_config=tool_config_from_mode("auto") # AUTO
)

# 대화 이력 확인
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("--")

user -> [{'text': '저는 57마리의 고양이를 키우고 있고, 각 44개의 손싸개를 가지고 있습니다. 손싸개는 총 몇 개 일까요?'}]
--
model -> [{'function_call': {'name': 'multiply', 'args': {'a': 57.0, 'b': 44.0}}}]
--
user -> [{'function_response': {'name': 'multiply', 'response': {'result': 2508.0}}}]
--
model -> [{'text': '손싸개는 총 2508개입니다. \n'}]
--


In [ ]:
# 대화 이력 삭제
chat.history.clear()

# 추론 실행
response = chat.send_message(
    "저는 57마리의 고양이를 키우고 있고, 각 44개의 손싸개를 가지고 있습니다. 손싸개는 총 몇 개 일까요?",
    tool_config=tool_config_from_mode("any", ["multiply"]) # ANY
)

# 대화 이력 확인
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("--")

user -> [{'text': '저는 57마리의 고양이를 키우고 있고, 각 44개의 손싸개를 가지고 있습니다. 손싸개는 총 몇 개 일까요?'}]
--
model -> [{'function_call': {'name': 'multiply', 'args': {'a': 57.0, 'b': 44.0}}}]
--
user -> [{'function_response': {'name': 'multiply', 'response': {'result': 2508.0}}}]
--
model -> [{'text': '고양이 손싸개는 총 2508개입니다. \n'}]
--


### 수동 함수 호출(Manual Function Calling)

In [ ]:
# 기온 정보 가져오기(더미)
def get_temperature(location: str):
    """get current temperature."""
    if location == "서울":
        return "20도"
    else:
        return "10도"

In [ ]:
# 함수를 사전 형태로 관리
functions = {
    "get_temperature": get_temperature,
}

In [ ]:
# 모델 초기화
model = genai.GenerativeModel(
    model_name="models/gemini-1.5-flash",
    tools=functions.values(),
)

In [ ]:
# 함수를 활용한 질의 응답
prompt = "현재 서울의 기온은?"
response = model.generate_content(prompt)
response.candidates[0].content.parts

[function_call {
  name: "get_temperature"
  args {
    fields {
      key: "location"
      value {
        string_value: "\354\204\234\354\232\270"
      }
    }
  }
}
]

In [ ]:
import google.ai.generativelanguage as glm

# 함수 호출
def call_function(function_call, functions):
    function_name = function_call.name
    function_args = function_call.args
    return functions[function_name](**function_args)

# function_responses 생성
function_responses = []
for part in response.parts:
    # 함수 사용을 선택했는지 확인
    if part.function_call:
        # 함수 호출 실행
        result = call_function(part.function_call, functions)

        # function_response 생성
        function_response = glm.Part(function_response=glm.FunctionResponse(
            name=part.function_call.name,
            response={"result": result}
        ))
        function_responses.append(function_response)
print(function_responses)

[function_response {
  name: "get_temperature"
  response {
    fields {
      key: "result"
      value {
        string_value: "20\353\217\204"
      }
    }
  }
}
]


In [ ]:
# 대화 이력 작성
messages = [
    {'role':'user',
     'parts': [prompt]},
    {'role':'model',
     'parts': response.candidates[0].content.parts},
    {'role':'user',
     'parts': function_responses}
]

# 질의 응답
response = model.generate_content(messages)
print(response.text)

현재 서울의 기온은 20도입니다. 



# 병렬 함수 호출(Parallel Function Calling)

In [ ]:
# 함수를 사용한 질의 응답
prompt = "현재 서울과 부산의 기온은?"
response = model.generate_content(prompt)
response.candidates[0].content.parts

[function_call {
  name: "get_temperature"
  args {
    fields {
      key: "location"
      value {
        string_value: "\354\204\234\354\232\270"
      }
    }
  }
}
, function_call {
  name: "get_temperature"
  args {
    fields {
      key: "location"
      value {
        string_value: "\353\266\200\354\202\260"
      }
    }
  }
}
]

In [ ]:
import google.ai.generativelanguage as glm

# 함수 호출
def call_function(function_call, functions):
    function_name = function_call.name
    function_args = function_call.args
    return functions[function_name](**function_args)

# function_responses 생성
function_responses = []
for part in response.parts:
    # 함수 사용을 선택했는지 확인
    if part.function_call:
        # 함수 호출 실행
        result = call_function(part.function_call, functions)

        # function_response 생성
        function_response = glm.Part(function_response=glm.FunctionResponse(
            name=part.function_call.name,
            response={"result": result}
        ))
        function_responses.append(function_response)
print(function_responses)

[function_response {
  name: "get_temperature"
  response {
    fields {
      key: "result"
      value {
        string_value: "20\353\217\204"
      }
    }
  }
}
, function_response {
  name: "get_temperature"
  response {
    fields {
      key: "result"
      value {
        string_value: "10\353\217\204"
      }
    }
  }
}
]


In [ ]:
# 대화 이력 작성
messages = [
    {'role':'user',
     'parts': [prompt]},
    {'role':'model',
     'parts': response.candidates[0].content.parts},
    {'role':'user',
     'parts': function_responses}
]

# 질의 응답
response = model.generate_content(messages)
print(response.text)

서울은 20도, 부산은 10도입니다. 

